Step 1: Set Up Basic Imports and LLM

In [1]:
# Required imports
from langchain_openai import ChatOpenAI  # or whatever model you use
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated


In [ ]:
# Set up a model:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.3)


Step 2: Define a Simple Tool — stardate_converter

In [3]:
from datetime import datetime

from datetime import datetime

def stardate_converter(earth_date: str) -> str:
    """Convert an Earth date to a fictional stardate."""
    base = datetime(2323, 1, 1)  # arbitrary base date for stardates
    dt = datetime.strptime(earth_date, "%Y-%m-%d")
    days = (dt - base).days
    stardate = 1000 + days * 0.036  # fake formula
    return f"Stardate {stardate:.2f}"


Step 3: Bind the Tool to the Model

In [7]:
llm_with_tools = llm.bind_tools([stardate_converter])

# Now the model can call stardate_converter() when it thinks it should!

Step 4: Define the State

In [9]:
from langchain_core.messages import AnyMessage

class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# This allows us to track all messages, and add_messages automatically merges the new ones.

Step 5: Define the Node

In [10]:
def starfleet_node(state: MessageState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# This node:

# Passes the full chat history to the LLM

# Returns its reply

# Lets LangGraph merge it into the message history

Step 6: Build the Graph

In [13]:
builder = StateGraph(MessageState)
builder.add_node("starfleet_node", starfleet_node)
builder.add_edge(START, "starfleet_node")
builder.add_edge("starfleet_node", END)

graph = builder.compile()

Step 7: Run It

In [14]:
initial_state = {
    "messages": [
        HumanMessage(content="Convert 2025-07-04 to a stardate.", name="Captain")
    ]
}

final_state = graph.invoke(initial_state)

for m in final_state["messages"]:
    m.pretty_print()


================================ Human Message =================================
Name: Captain

Convert 2025-07-04 to a stardate.
================================== Ai Message ==================================
Tool Calls:
  stardate_converter (call_jlbUTCucC2XEm3W2eFpxf7Rd)
 Call ID: call_jlbUTCucC2XEm3W2eFpxf7Rd
  Args:
    earth_date: 2025-07-04
